In [3]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://busan.scourt.go.kr/dcboard/new/DcNewsViewAction.work?&gubun=44&cbub_code=000410&searchWord=&pageIndex=1'
params = {
  'seqnum':1
  }

resp = requests.get(base_url, params=params)
soup = BeautifulSoup(resp.content)
tbody_list = []
tbody = soup.find('tbody')
print(tbody)

<tbody>
<tr>
<th scope="row">제목</th>
<td class="title" colspan="5">특허법원 2005. 10. 21. 선고 2005허5259 판결 [등록무효(의)]</td>
</tr>
<tr>
<th scope="row">작성자</th>
<td>특허법원 [대전]</td>
<th scope="row">작성일</th>
<td>2005.10.21</td>
<th scope="row">조회</th>
<td>25215</td>
</tr>
<!-- 첨부파일 Start -->
<tr>
<th scope="row">첨부파일</th>
<td class="board_head03" colspan="5">
<a href="javascript:download('1129870139007_134859.pdf','2005허5259.pdf')" title="2005허5259.pdf">2005허5259.pdf</a>
                                    
                                    
                                    
                                    
                                     
                                    </td>
</tr>
<!-- 첨부파일 End  -->
<tr>
<td colspan="6">
<div class="view_content">
<p>사건 : 특허법원 2005. 10. 21. 선고 2005허5259 판결 [등록무효(의)] - 족구공 디자인 사건</p>
<p> </p>
<p>판시사항 :  가. 족구공과 배구공이 유사한 물품에 해당하는지 여부(적극)</p>
<p>                나. 공의 표면에 형성된 모양이 서로 유사하기는 하나 그 모양을 이루는 가죽 조각에 채색</p>
<p>                     되어 있는 색채